In [50]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import os
import pandas as pd


print("============================================================================================================")
print("유튜브 댓글 추출")
print("============================================================================================================")
keyword = input("1. 검색할 키워드를 입력하세요: ")
num_videos = int(input("2. 크롤링할 영상 개수를 입력하세요: "))
num_comments = int(input("3. 각 영상에서 추출할 댓글 개수를 입력하세요: "))
f_dir = input("4.결과 파일을 저장할 폴더명만 쓰세요(예:c:\\data\\):")
# keyword = "떡볶이"
# num_videos = 3
# num_comments = 2
# f_dir = "c:\\data\\"

n = time.localtime()
s = "%04d-%02d-%02d-%02d-%02d-%02d" % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir + s + "-" + keyword)
os.chdir(f_dir + s + "-" + keyword)

ft_name=f_dir+s+'-'+keyword+'\\'+s+'-'+keyword+'.txt'
fc_name=f_dir+s+'-'+keyword+'\\'+s+'-'+keyword+'.csv'
fx_name=f_dir+s+'-'+keyword+'\\'+s+'-'+keyword+'.xls'

path = "C:\Temp\chromedriver_win32\chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get("http://youtube.com")

# 유튜브 검색 입력란 찾기
search_box = driver.find_element(By.NAME, "search_query")
# 검색어 입력
search_box.send_keys(keyword)
search_box.send_keys(Keys.RETURN)
time.sleep(2)

# 영상 링크 저장할 리스트 초기화
video_links = []

full_html = driver.page_source

soup = BeautifulSoup(full_html, 'html.parser')

video_renderers = soup.find_all("ytd-video-renderer", class_="style-scope ytd-item-section-renderer")

for video_renderer in video_renderers:
    thumbnail = video_renderer.find("ytd-thumbnail", class_="style-scope ytd-video-renderer")
    thumbnail_link = thumbnail.find("a", class_="yt-simple-endpoint inline-block style-scope ytd-thumbnail").get("href")
    video_links.append(thumbnail_link)
    
    
video_count = 0
writers = []
comments = []
dates = []
urls = []
video_num_list = []
comment_num_list = []

# num_videos 개수만큼 영상 크롤링
for i, video_link in enumerate(video_links[:num_videos]):
    new_link = "http://youtube.com" + video_link
    driver.get(new_link)
    time.sleep(2)
    video_count += 1
    comment_count = 0
    # 페이지 스크롤 다운
    while len(comments) < num_comments * num_videos:
        # 현재 페이지의 HTML 소스 가져오기
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")

        # 댓글 추출
        comment_elements = soup.find_all("ytd-comment-thread-renderer")
        for k in comment_elements:
            if comment_count >= num_comments:
                break
            kk = k.find('ytd-comment-renderer').find('div', id='body').find('div', id='main')
            writer = kk.find('div', id='header').find('div', id='header-author').find('h3').get_text()
            writers.append(writer.strip())
            print("작성자: ", writer.strip())

            date = kk.find('div', id='header').find('div', id='header-author').find('yt-formatted-string', class_='published-time-text style-scope ytd-comment-renderer').get_text()
            dates.append(date.strip())
            print("작성일자: ", date.strip())

            comment = kk.find('div', id='comment-content').get_text()
            comments.append(comment.strip())
            print("내용: ", comment.strip())

            urls.append(new_link)
            comment_count += 1
            video_num_list.append(video_count)
            comment_num_list.append(comment_count)
            
        if comment_count >= num_comments:
            break

        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(2)


df = pd.DataFrame()

df['영상번호'] = video_num_list
df['댓글번호'] = comment_num_list
df['작성자'] = writers
df['작성일자'] = dates
df['내용'] = comments
df['url'] = urls

with open(ft_name, 'w', encoding='utf-8') as file:
    for i in range(len(urls)):
        file.write(str(video_num_list[i]) + '번째 영상 ' + str(comment_num_list[i]) + '번째 댓글입니다. =============')
        file.write('1. 작성자: ' + writers[i] + '\n')
        file.write('2. 작성일자: ' + dates[i] + '\n')
        file.write('3. 내용: ' + comments[i] + '\n')
        file.write('4. 영상url: ' + urls[i] + '\n')
        
        file.write('\n')

df.to_csv(fc_name, encoding="utf-8-sig", index=False)
df.to_excel(fx_name, index=False)
print("추출이 완료되었습니다.")

유튜브 댓글 추출
작성자:  삐용삐용
작성일자:  3년 전(수정됨)
내용:  백선생님께서 이 글을 보실지못보실지는 모르겠지만
선생님 덕분에 혼자 서울살이 잘 견뎠고, 잘 견디고 있어요. 혼자 시켜먹는것도 한계가 있고, 서울은 음식값이 비싸기도 하고...원룸에서도 쉽게 해먹을수있게 가르쳐주셔서 정말 덜 외로웠어요. 배부르면 덜외롭잖아요. 고맙습니다. 항상 응원합니다.


간략히


자세히 보기
작성자:  tae haha
작성일자:  1년 전(수정됨)
내용:  3:34  대파
4:01  양배추
4:07  양파
4:33  물 1L
4:49  진간장 1/3컵
4:58  황설탕 1/3컵
5:03  고추장 1/3컵
5:12  고춧가루 2/3컵


간략히


자세히 보기
작성자:  [윤이련]50년 요리비결
작성일자:  1년 전
내용:  가을비가 축축히 내리는군요
새끼 넷 을 키우고 살다보니
떡뽁기는 세상쉬운 나의 한손음식
젊은시절 장사할때 가장쉽고
애들 배채우든 간식거리 떡뽁기 라뽁기 ㅎㅎ
우리애들 고만고만 책가방 넷일때 바빴든 그세월이 생각납니다


간략히


자세히 보기
작성자:  Gregory P.
작성일자:  1년 전
내용:  수많은 레시피로 따라해 봤지만 이번이 가장 맛있는 떡볶이 였습니다. 덕분에 가족들도 넘넘 좋아하네요. 감사합니다. ^^


간략히


자세히 보기
작성자:  상해기SangHyuk
작성일자:  5시간 전
내용:  ■매주 일,월,수,금 오후 9시15분 실시간 최초공개로 영상이 업로드 됩니다
 
해콩이 여러분 안녕하세요 상해기입니다.
오늘은 경상북도 안동 안동구시장 시장먹방 입니다.
 
오늘은 특별게스트로 제 조카 이대은군과 함께 해 봤습니다
이번이 돈까스먹방 그리고 작년 관문시장 시장먹방에 이은 
3번째 합동방송인데요. 아주 미친폼을 보여줬습니다
 
재밌게 봐 주세요!
영상이 괜찮으셨다면 구독 좋아요 알림설정 부탁드릴게요 
오늘도 함께 해 주셔서 감사합니다 
 
*시장정보
시장: 안동구시장
위치 : 경북 안동시 서부동 185
 
1. 원조보리밥
